### Preprocessing ###
Dieses Jupyter Notebook behandelt das Preprocessing der 100 Stichproben, und schliesslich das Anwenden des Scripts auf den Gesamtkorpus. Dabei wird wie folgt vorgegangen:
1. Aufteilen der Samples in ihre jeweilichen Epoche x Hauptklasse-Schicht
2. Spezifisches Preprocessing mit diversen Unterschritten:
    - Wortdistributionen (für Stopwords)
    - Wortähnlichkeitsanalyse (für verschiedene Schreibweisen)
    - Unicode-Normalisierung (für archaische Buchstaben)
    - Leerzeichen-Entfernung (für unnötige Leerzeichen)
    - Lower-Casing (für bessere Maschinenlesbarkeit)
    - Lemmatisierung (für bessere Ergebnisse beim Modell-Training)
    - Tokenization
    - Abkürzungsmanagement
    - Mehrwortausdruck-Management
    - Frequenzfilterung
    - Zeichensetzung (Entfernung von unnötigen Zeichen, Punkte, Doppelpunkte etc. bleiben erhalten)
3. Anwenden des Preprocessing-Scripts auf die jeweilige Epoche x Genre-Schicht im Gesamtkorpus
4. Überprüfen der Ergebnisse mit BERTopic pro Epoche x Genre-Schicht
5. Wiederholen der Schritte für weitere Epochen x Genres
6. Durchlauf des Gesamtkorpus mit BERTopic

## 1. Aufteilen der Samples in ihre jeweiligen Epoche x Hauptklasse-Schicht ##
In diesem Script werden die relevanten listen erstellt, die die Texte der Stichproben enthalten.

In [3]:
import sqlite3

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

outputs = cur.execute("SELECT DISTINCT epochen_genres_strata FROM samples").fetchall()

for output in outputs:
    print(output[0])

print(len(outputs))

con.close()


Barock x Gebrauchsliteratur
Aufklärung x Belletristik
Renaissance x Zeitung
Barock x Zeitung
Revolutionsjahre x Zeitung
Moderne x Wissenschaft
Aufklärung x Wissenschaft
Aufklärung x Gebrauchsliteratur
Industrielle Revolution x Wissenschaft
Frühes 19. Jahrhundert x Gebrauchsliteratur
Barock x Belletristik
Renaissance x Belletristik
Spätmittelalter x Belletristik
Aufklärung x Zeitung
Frühes 19. Jahrhundert x Zeitung
Moderne x Gebrauchsliteratur
Industrielle Revolution x Belletristik
Industrielle Revolution x Gebrauchsliteratur
Frühes 19. Jahrhundert x Belletristik
Revolutionsjahre x Belletristik
Spätmittelalter x Gebrauchsliteratur
Industrielle Revolution x Zeitung
Moderne x Zeitung
Revolutionsjahre x Gebrauchsliteratur
Renaissance x Gebrauchsliteratur
Renaissance x Wissenschaft
Frühes 19. Jahrhundert x Wissenschaft
Revolutionsjahre x Wissenschaft
Barock x Wissenschaft
Moderne x Belletristik
30


In [28]:
import sqlite3
import pprint as pp

con = sqlite3.connect("Datenbank.db")
cur = con.cursor()

epoch_dict = {}

data = cur.execute("SELECT text, epochen_genres_strata FROM samples ORDER BY epochen_genres_strata").fetchall()

for text, genre in data:
    if genre not in epoch_dict:
        epoch_dict[genre] = []
    epoch_dict[genre].append(text)

con.close()

Für das Preprocessing werden unterschiedliche libraries verwendet:
- re: Um Buchstaben, Leerzeichen, Absatzzeichen zu ersetzen oder zu löschen.
- unicodedata: Für die Normalisierung von Spezialcharakteren
- spaCy: Für die Tokenisierung und Lemmatisierung
- gensim: Für die Mehrwortausdrücke
- rapidfuzz: Für die Wortähnlichkeitsanalyse
- collections: Frequenzfilterung
- und eventuell mehr, wie notwendig

Einige dieser libraries müssen heruntergeladen werden (spaCy, gensim, rapidfuzz).

In [30]:
%pip install spaCy gensim rapidfuzz

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 1.6 MB/s eta 0:00:00
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/11.8 MB 3.5 MB/s eta 0:00:04
   - -------------------------------------- 0.4/11.8 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.8 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.6/11.8 MB 2.2 M

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. a) Revolutionsjahre x Zeitung ##

Als erstes wird die grösste Kategorie behandelt: Revolutionsjahre x Zeitung, mit 15 Samples.

Zunächst wird ein Beispiel eines Textes angeschaut.

In [31]:
print(epoch_dict["Revolutionsjahre x Zeitung"][0])

        Augsburger Allgemeine Zeitung.   
 Mit allerhöchsten Privilegien.     
 Donnerstag   
 Nr. 2.   
 2 Januar 1840.         
 
 
 
   Vereinigte Staaten von Nordamerika.   
 Am 27 Dec. lief in Havre das Paketboot Baltimore mit Briefen und Journalen aus New-York bis zum 5 Dec. ein, aber ebenfalls  ohne  die Präsidentenbotschaft. Der Congreß war am 2 Dec. wirklich zusammen getreten; indeß erhob sich über die Organisation des Repräsentantenhauses eine Präliminarverhandlung, welche die Versammlung bis zum Abgange des Paketboots beschäftigte. Da es Regel ist, daß die Botschaft erst nach erfolgter vollständiger Bildung beider Häuser in den Congreß gebracht werden kann, so erklärt sich hieraus die verspätete Vorlegung dieses Documents. Die Debatte entspann sich über die Gültigkeit der Wahl gewisser Mitglieder für den Staat New-Jersey, deren respectives Recht im Congreß zu sitzen durch den Clerk des Hauses nicht entschieden werden konnte. Gegen zwanzig Mitglieder hielten sehr heftige und 

Einige erste Beobachtungen:

- Character Encoding: Spezialcharakter ☉, ✝, △
- Abkürzungen: diverse, z.B. kaiserl., Hr., Sr. Maj., Thlr., gr., fl, Rthls.
- Archaische Ausdrücke: diverse, z.B. Bewirthung, sey/seyn, publicirt, Lesecirkel
- Mehrsprachigkeit (Deutsch, Französisch, englische Ausdrücke)

a.1. Unicode-Normalisierung

Duch die Unicode-Normalisierung wird sichergestellt, dass alle Buchstaben gleich ausgelesen werden können. Dies ist insbesondere wichtig bei der Tokenisierung und Lemmatisierung später: Inkosistente Repräsentationen von Wörtern können zu Fehlern führen oder die Wörter nicht erkannt werden. Es wurde die Normalform "NFKC" gebraucht, damit potentielle Ligaturen in die Doppelform umgewandelt werden. 

In [42]:
import unicodedata

for text in epoch_dict["Revolutionsjahre x Zeitung"]:
    unicodedata.normalize("NFKC", text)

print(epoch_dict["Revolutionsjahre x Zeitung"][0])

        Augsburger Allgemeine Zeitung.   
 Mit allerhöchsten Privilegien.     
 Donnerstag   
 Nr. 2.   
 2 Januar 1840.         
 
 
 
   Vereinigte Staaten von Nordamerika.   
 Am 27 Dec. lief in Havre das Paketboot Baltimore mit Briefen und Journalen aus New-York bis zum 5 Dec. ein, aber ebenfalls  ohne  die Präsidentenbotschaft. Der Congreß war am 2 Dec. wirklich zusammen getreten; indeß erhob sich über die Organisation des Repräsentantenhauses eine Präliminarverhandlung, welche die Versammlung bis zum Abgange des Paketboots beschäftigte. Da es Regel ist, daß die Botschaft erst nach erfolgter vollständiger Bildung beider Häuser in den Congreß gebracht werden kann, so erklärt sich hieraus die verspätete Vorlegung dieses Documents. Die Debatte entspann sich über die Gültigkeit der Wahl gewisser Mitglieder für den Staat New-Jersey, deren respectives Recht im Congreß zu sitzen durch den Clerk des Hauses nicht entschieden werden konnte. Gegen zwanzig Mitglieder hielten sehr heftige und 

a.2. Leerzeichen

Ziel dieses Schrittes ist es, einen Lauftext mit dem Inhalt zu generieren. Dabei werden Absätze und multiple Leerzeichen entfernt.

In [ ]:
for key, texts in epoch_dict.items():
    # Absatzentfernung
    epoch_dict[key] = [text.replace("\n", "") for text in texts]

    # Leerzeichenentfernung
    epoch_dict[key] = [" ".join(text.split()) for text in texts]


Augsburger Allgemeine Zeitung. Mit allerhöchsten Privilegien. Donnerstag Nr. 2. 2 Januar 1840. Vereinigte Staaten von Nordamerika. Am 27 Dec. lief in Havre das Paketboot Baltimore mit Briefen und Journalen aus New-York bis zum 5 Dec. ein, aber ebenfalls ohne die Präsidentenbotschaft. Der Congreß war am 2 Dec. wirklich zusammen getreten; indeß erhob sich über die Organisation des Repräsentantenhauses eine Präliminarverhandlung, welche die Versammlung bis zum Abgange des Paketboots beschäftigte. Da es Regel ist, daß die Botschaft erst nach erfolgter vollständiger Bildung beider Häuser in den Congreß gebracht werden kann, so erklärt sich hieraus die verspätete Vorlegung dieses Documents. Die Debatte entspann sich über die Gültigkeit der Wahl gewisser Mitglieder für den Staat New-Jersey, deren respectives Recht im Congreß zu sitzen durch den Clerk des Hauses nicht entschieden werden konnte. Gegen zwanzig Mitglieder hielten sehr heftige und – was sich in den Vereinigten Staaten von selbst v

a.3. Entfernung unnötiger Zeichen

a.4. Lowercase

a.5. Abkürzungsmanagement

a.6. Tokenisierung

a.7. Mehrfachwortausdrücke

a.8. Wortähnlichkeitsanalyse

a.9. Lemmatisierung 

a.10. Stopword-Filterung

a.11. Frequenzfilterung